---
## BigDL-Nano Resnet example on CIFAR10 dataset
---
This example illustrates how to apply bigdl-nano optimizations on a image recognition case based on pytorch-lightning framework. The basic image recognition module is implemented with Lightning and trained on [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) image recognition Benchmark dataset.

In [1]:
from time import time
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy
from bigdl.nano.pytorch.trainer import Trainer
from bigdl.nano.pytorch.vision import transforms

/root/anaconda3/envs/nanoPytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### CIFAR10 Data Module
---
Import the existing data module from bolts and modify the train and test transforms.
You could access [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) for a view of the whole dataset.


In [2]:
def prepare_data(data_path, batch_size, num_workers):
    train_transforms = transforms.Compose(
        [
            transforms.RandomCrop(32, 4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            cifar10_normalization()
        ]
    )

    test_transforms = transforms.Compose(
        [
            transforms.ToTensor(),
            cifar10_normalization()
        ]
    )
    cifar10_dm = CIFAR10DataModule(
        data_dir=data_path,
        batch_size=batch_size,
        num_workers=num_workers,
        train_transforms=train_transforms,
        test_transforms=test_transforms,
        val_transforms=test_transforms
    )
    return cifar10_dm

### Resnet
---
Modify the pre-existing Resnet architecture from TorchVision. The pre-existing architecture is based on ImageNet images (224x224) as input. So we need to modify it for CIFAR10 images (32x32).

In [3]:
def create_model():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

### Lightning Module
---
Check out the [configure_optimizers](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#configure-optimizers) method to use custom Learning Rate schedulers. The OneCycleLR with SGD will get you to around 92-93% accuracy in 20-30 epochs and 93-94% accuracy in 40-50 epochs. Feel free to experiment with different LR schedules from https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

In [4]:
class LitResnet(LightningModule):
    def __init__(self, learning_rate=0.05, steps_per_epoch=45000 , batch_size=32):
        super().__init__()

        self.save_hyperparameters()
        self.model = create_model()

    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.learning_rate,
            momentum=0.9,
            weight_decay=5e-4,
        )
        steps_per_epoch = self.hparams.steps_per_epoch // self.hparams.batch_size
        scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}

In [5]:
seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 64
NUM_WORKERS = 0
data_module = prepare_data(PATH_DATASETS, BATCH_SIZE, NUM_WORKERS)
DEV_RUN = bool(os.environ.get('DEV_RUN', False))
EPOCHS = 30

Global seed set to 7


### Train
Use Trainer from bigdl.nano.pytorch.trainer for BigDL-Nano pytorch.

This Trainer extends PyTorch Lightning Trainer by adding various options to accelerate pytorch training.

```
    :param num_processes: number of processes in distributed training. default: 4.
    :param use_ipex: whether we use ipex as accelerator for trainer. default: True.
    :param cpu_for_each_process: A list of length `num_processes`, each containing a list of
            indices of cpus each process will be using. default: None, and the cpu will be
            automatically and evenly distributed among processes.
```
The next few cells show examples of different parameters.
#### Single Process
---

In [6]:
model = LitResnet(learning_rate=0.05)
model.datamodule = data_module
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints/", save_top_k=1, monitor="val_loss", filename="renet18_single_none")
basic_trainer = Trainer(num_processes = 1,
                  use_ipex = False,
                  progress_bar_refresh_rate=10,
                  max_epochs=EPOCHS,
                  fast_dev_run=DEV_RUN,
                  logger=TensorBoardLogger("lightning_logs/", name="basic"),
                  callbacks=[LearningRateMonitor(logging_interval="step"), checkpoint_callback])
start = time()
basic_trainer.fit(model, datamodule=data_module)
basic_fit_time = time() - start
outputs = basic_trainer.test(model, datamodule=data_module)
basic_acc = outputs[0]['test_acc'] * 100
basic_trainer.save_checkpoint("checkpoints/model.ckpt")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.696    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 7
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.902899980545044, 'test_loss': 0.2910485565662384}
--------------------------------------------------------------------------------


### Single Process with IPEX
---

In [7]:
model = LitResnet(learning_rate=0.05)
model.datamodule = data_module
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints/", save_top_k=1, monitor="val_loss", filename="renet18_single_ipex", save_weights_only=True)
single_ipex_trainer = Trainer(num_processes=1,
                        use_ipex = True,
                        distributed_backend="subprocess",
                        progress_bar_refresh_rate=10,
                        max_epochs=EPOCHS,
                        fast_dev_run=DEV_RUN,
                        logger=TensorBoardLogger("lightning_logs/", name="single_ipex"),
                        callbacks=[LearningRateMonitor(logging_interval="step"), checkpoint_callback])
start = time()
single_ipex_trainer.fit(model, datamodule=data_module)
single_ipex_fit_time = time() - start
outputs = single_ipex_trainer.test(model, datamodule=data_module)
single_ipex_acc = outputs[0]['test_acc'] * 100

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory checkpoints/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.696    Total estimated model params siz

[WARNING] "import intel_pytorch_extension" will be deprecated in future releases. Please use "import torch_ipex" instead.


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 7


Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.899399995803833, 'test_loss': 0.2945179343223572}
--------------------------------------------------------------------------------


### Multiple Processes with IPEX
---

In [9]:
model = LitResnet(learning_rate=0.1, batch_size=64)
model.datamodule = data_module
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints/", save_top_k=1, monitor="val_loss", filename="renet18_multi_ipex", save_weights_only=True)
multi_ipex_trainer = Trainer(num_processes=2,
                       use_ipex=True,
                       distributed_backend="subprocess",
                       progress_bar_refresh_rate=10,
                       max_epochs=EPOCHS,
                       fast_dev_run=DEV_RUN,
                       logger=TensorBoardLogger("lightning_logs/", name="multi_ipx"),
                       callbacks=[LearningRateMonitor(logging_interval="step"), checkpoint_callback])
start = time()
multi_ipex_trainer.fit(model, datamodule=data_module)
multi_ipex_fit_time = time() - start
outputs = multi_ipex_trainer.test(model, datamodule=data_module)
multi_ipex_acc = outputs[0]['test_acc'] * 100

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory checkpoints/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py:75: LightningDeprecationWarning: Argument `num_nodes` in `DDPSpawnPlugin` is deprecated in v1.4, and will be removed in v1.6. Notice that it will be overriden by the trainer setting.
  "Argument `num_nodes` in `DDPSpawnPlugin` is deprecated in v1.4, and will be removed in v1.6. "
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py:81: LightningDeprecationWarning: Argument `sync_batchnorm` in `DDPSpawnPlugin` is deprecated in v1.4, and will be removed in v1.6. Notice that it will be overriden by the trainer setting.
  "A

[WARNING] "import intel_pytorch_extension" will be deprecated in future releases. Please use "import torch_ipex" instead.
Epoch 0:   0%|          | 0/782 [00:00<00:00, 3953.16it/s]    

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 7
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 7
/home/mingzhi/anaconda3/

Epoch 0:  81%|████████  | 630/782 [01:31<00:21,  6.92it/s, loss=1.54, v_num=1]
Validating: 0it [00:00, ?it/s]
Validating:   0%|          | 0/157 [00:00<?, ?it/s]

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:125: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "


evaluate


Global seed set to 7
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/2
Global seed set to 7
initializing ddp: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=gloo
All DDP processes registered. Starting ddp with 2 processes
----------------------------------------------------------------------------------------------------

/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


[WARNING] "import intel_pytorch_extension" will be deprecated in future releases. Please use "import torch_ipex" instead.
Testing: 100%|██████████| 157/157 [00:06<00:00, 25.05it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8917999863624573, 'test_loss': 0.3283665180206299}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 157/157 [00:06<00:00, 22.51it/s][WARNING] "import intel_pytorch_extension" will be deprecated in future releases. Please use "import torch_ipex" instead.
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8917999863624573, 'test_loss': 0.3283665180206299}
--------------------------------------------------------------------------------



/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py:288: UserWarning: cleaning up ddp environment...
  rank_zero_warn("cleaning up ddp environment...")
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py:288: UserWarning: cleaning up ddp environment...
  rank_zero_warn("cleaning up ddp environment...")
/home/mingzhi/anaconda3/envs/nanoWithPytorch/lib/python3.7/site-packages/pytorch

In [10]:
template = """
|      Precision    | Fit Time(s)       | Accuracy(%) |
|        Basic      |       {:5.2f}       |    {:5.2f}    |
|  Single With Ipex |       {:5.2f}       |    {:5.2f}    |
| Multiple With Ipex|       {:5.2f}       |    {:5.2f}    |
"""
summary = template.format(
    basic_fit_time, basic_acc,
    single_ipex_fit_time, single_ipex_acc,
    multi_ipex_fit_time, multi_ipex_acc
)
print(summary)


|      Precision    | Fit Time(s)       | Accuracy(%) |
|        Basic      |       4559.45       |    90.29    |
|  Single With Ipex |       4380.92       |    89.94    |
| Multiple With Ipex|       2633.85       |    89.18    |

